<a href="https://colab.research.google.com/github/bekircan4721/Bekircan_arac-/blob/main/Correlations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr, spearmanr, t

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# reading datasets
data_shark = pd.read_csv("https://raw.githubusercontent.com/bekircan4721/Bekircan_arac-/main/attacks.csv", encoding="unicode_escape")
data_temperature = pd.read_table("https://www.ncei.noaa.gov/data/oceans/woa/DATA_ANALYSIS/3M_HEAT_CONTENT/DATA/basin/pentad/pent_h22-w0-2000m.dat", delim_whitespace=True)

# filtering by year
data_shark = data_shark[data_shark["Year"].notnull()]
data_shark["Year"] = data_shark["Year"].astype(int)

# yearly ocean temperatures
temperature_by_year = data_temperature[["YEAR", "WO"]].copy()
temperature_by_year.columns = ["Year", "Ocean_Temperature"]
temperature_by_year["Year"] = temperature_by_year["Year"].round().astype(int)

# Get shark attacks by year
shark_by_year = data_shark.groupby("Year").size().reset_index(name="Attack_Count")

# Merge datasets based on Year
merged = pd.merge(shark_by_year, temperature_by_year, on="Year", how="left")
merged = merged.dropna(subset=["Ocean_Temperature"])

# Pearson Correlation
r_pearson, _ = pearsonr(merged['Ocean_Temperature'], merged['Attack_Count'])
df = len(merged) - 2 # we have 2 columns thats why len - 2
t_stat = r_pearson * np.sqrt(df) / np.sqrt(1 - r_pearson**2)
p_value_pearson = 2 * t.sf(np.abs(t_stat), df)

# Spearman Correlation
r_value_spearman, p_value_spearman = spearmanr(merged['Ocean_Temperature'], merged['Attack_Count'])

# Print results
print(f"-----  Pearson Correlation ----- ")
print(f"r (Pearson): {r_pearson:.3f}")
print(f"t-value: {t_stat:.3f}")
print(f"Degrees of Freedom: {df}")
print(f"p-value: {p_value_pearson:.4f}\n")

print(f"----- Spearman Correlation ----- ")
print(f"r (Spearman): {r_value_spearman:.3f}")
print(f"p-value: {p_value_spearman:.4f}")


<ipython-input-1-6c082dc4e4ef>:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data_temperature = pd.read_table("https://www.ncei.noaa.gov/data/oceans/woa/DATA_ANALYSIS/3M_HEAT_CONTENT/DATA/basin/pentad/pent_h22-w0-2000m.dat", delim_whitespace=True)
<ipython-input-1-6c082dc4e4ef>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shark["Year"] = data_shark["Year"].astype(int)


-----  Pearson Correlation ----- 
r (Pearson): 0.674
t-value: 7.062
Degrees of Freedom: 60
p-value: 0.0000

----- Spearman Correlation ----- 
r (Spearman): 0.567
p-value: 0.0000
